In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import cv2
import seaborn as sns
from skimage.filters import sobel
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers,models



# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [45]:
#resizing image
size = 128

In [46]:
#Capture images and labels into array
#Start by creating empty lists

train_images = []
train_labels = []

# for directory_path in glob.glob("/kaggle/input/malaria-detection-dataset/Dataset/Train/*"):
#     label = directory_path.split("/")[-1]
#     #print(label)
#     for img_path in glob.glob(os.path.join(directory_path,"*.jpg")):
#         print(img_path)


for directory_path, _, filenames in os.walk('/kaggle/input/malaria-detection-dataset/Dataset/Train/'):
    label = directory_path.split("/")[-1]
    #print(label)
    for filename in filenames:
        img_path = os.path.join(directory_path, filename)
        #print(img_path)
        
        img = cv2.imread(img_path,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(size,size))
        #img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [47]:
#Do the same for test images

test_images = []
test_labels = []

for directory_path, _, filenames in os.walk('/kaggle/input/malaria-detection-dataset/Dataset/Test/'):
    label = directory_path.split("/")[-1]
    #print(label)
    for filename in filenames:
        img_path = os.path.join(directory_path, filename)
        #print(img_path)
        
        img = cv2.imread(img_path,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(size,size))
        #img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [48]:
#Encode labels from text (folder_names) to integers
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [49]:
#Split data into test and train datasets

x_train,y_train,x_test,y_test = train_images,train_labels_encoded,test_images,test_labels_encoded

#Normalize pixel value between 0 and 1

x_train,x_test = x_train/255.0, x_test/255.0

In [106]:
#CNN model building

cnn = models.Sequential([
    #1st layer or Input layer
    
    layers.Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', input_shape = (128,128,3)),
    #Pooling
    layers.MaxPool2D((2,2)),
   
    
    #2nd layer
    
    layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'),
    layers.MaxPool2D((2,2)),
    
    
    
    #3rd layer
    
    layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'),
    layers.MaxPool2D((2,2)),
    
    
    #4th layer
    
    layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'),
    layers.MaxPool2D((2,2)),
    
    
    #5th layer
    
    layers.Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'),
    layers.MaxPool2D((2,2)),
    
    
    
    #Flattening
    
    layers.Flatten(),
    
    
    #Last connection
    
    layers.Dense(512, activation = 'relu'),

    
    #For Binary Classification
    
    layers.Dense(1, activation = 'sigmoid')
])

In [107]:
#compiling

cnn.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [108]:
#Fitting model
history = cnn.fit(x_train,
        y_train,
        epochs = 12)

In [109]:
cnn.summary()

In [110]:
#Reshape test dataset


#Prediction
test_prediction = cnn.predict(x_test)

#test_prediction
#test_labels.size
#x_train.shape

#inverse le transform to get original label back

#test_pre = le.inverse_transform(test_prediction)


In [111]:
test_pre = []
for i in test_prediction:
    if(i >= 0.5):
        test_pre.append(1)
    else:
        test_pre.append(0)
#test_pre        



In [112]:
test = le.inverse_transform(test_pre)
test

In [113]:
accuracy = cnn.evaluate(x_test,y_test)
print("Accuracy: ",accuracy)
#Accuracy:  [0.12909041345119476, 0.9477611780166626]

In [114]:
#print overall accuracy

from sklearn import metrics
print("Accuracy = ",metrics.accuracy_score(test_labels,test))
#Accuracy =  0.9477611940298507

In [124]:
import random
n = random.randint(0,x_test.shape[0]-1)
img = x_test[n]
plt.imshow(img);

#predict

input_img = np.expand_dims(img,axis = 0)
# img_new = np.reshape(img,(input_img.shape[0],-1))
img_prediction = cnn.predict(input_img)
if(img_prediction >= 0.5):
    img_prediction = 1
else:
    img_prediction = 0
img_prediction = le.inverse_transform([img_prediction])
print("The prediction for this image is : ", img_prediction)
print("The actual label for this image is : ", test_labels[n])